(how-to-select-algorithms)=
# How to select a local optimizer
## Why choosing the right optimizer is difficult

Optimization without the knowldge of the properties of the problem we are trying to solve is difficult. 

The only algorithms that are guaranteed to solve all problems are grid search or other algorithms that evaluate the criterion function almost everywhere in the parameter space.

If you have more than a hand full of parameters, these methods would take too long.

Thus, you have to know the properties of your optimization problem and have knowledge about different optimization algorithms in order to choose the right algorithm for your problem.

## How to approach algorithm selection

We recommend the following workflow to choose algorithms:
  1. Based on the theoretical properties of your problem, narrow the set of algorithms to experiment with to 2-3 algorithms.
  2. Run the algorithms for smaller number of iterations. As a rule of thumb, use 10*`n_params` iterations or function evaluations.
  3. Compare the results in a criterion plot.
  4. Re run the optimization algorithm with the best results until a convergence criterion is achieved.
  
## Step 1: Choosing candidates

The below decision tree offers a practical guide on how to narrow down the set of algorithms to experiment with, based on the theoretical properties of your problem:
```{mermaid}
graph LR
    classDef highlight fill:#FF4500;
    A["Do you have<br/>nonlinear constraints?"] -- yes --> B["differentiable?"]
    B["differentiable?"] -- yes --> C["'ipopt', 'nlopt_slsqp', 'scipy_trust_constr'"]
    B["differentiable?"] -- no --> D["'scipy_cobyla', 'nlopt_cobyla'"]

    A["Do you have<br/>nonlinear constraints?"] -- no --> E["Can you exploit<br/>a least-squares<br/>structure?"]
    E["Can you exploit<br/>a least-squares<br/>structure?"] -- yes --> F["differentiable?"]
    E["Can you exploit<br/>a least-squares<br/>structure?"] -- no --> G["differentiable?"]

    F["differentiable?"] -- yes --> H["'scipy_ls_lm', 'scipy_ls_trf', 'scipy_ls_dogleg'"]
    F["differentiable?"] -- no --> I["'nag_dflos', 'pounders', 'tao_pounders'"]

    G["differentiable?"] -- yes --> J["'scipy_lbfgsb', 'nlopt_lbfgsb', 'fides'"]
    G["differentiable?"] -- no --> K["'nlopt_bobyqa', 'nlopt_neldermead', 'neldermead_parallel'"]
```



In [ ]:
import numpy as np

import optimagic as om

In [ ]:
def trid_scalar(x):
    """Implement Trid function: https://www.sfu.ca/~ssurjano/trid.html."""
    return ((x - 1) ** 2).sum() - (x[1:] * x[:-1]).sum()


def trid_gradient(x):
    """Calculate gradient of trid function."""
    l1 = np.insert(x, 0, 0)
    l1 = np.delete(l1, [-1])
    l2 = np.append(x, 0)
    l2 = np.delete(l2, [0])
    return 2 * (x - 1) - l1 - l2


results = {}
for algo in ["scipy_lbfgsb", "nlopt_lbfgsb", "fides"]:
    results[algo] = om.minimize(
        fun=trid_scalar,
        jac=trid_gradient,
        params=np.arange(20),
        algorithm=algo,
        algo_options={"stopping_maxfun": 8, "stopping_maxiter": 8},
    )

In [ ]:
fig = om.criterion_plot(results, max_evaluations=8)
fig.show(renderer="png")

In [ ]:
best_x = results["nlopt_lbfgsb"].params
results["nlopt_lbfgsb_complete"] = om.minimize(
    fun=trid_scalar,
    jac=trid_gradient,
    params=best_x,
    algorithm="nlopt_lbfgsb",
)

In [ ]:
fig = om.criterion_plot(results)
fig.show(renderer="png")